#  Graphs and Graph Algorithms 2



## The Knight’s Tour Problem

Another classic problem that we can use to illustrate a second common graph algorithm is called the “knight’s tour.” The knight’s tour puzzle is played on a chess board with a single chess piece, the knight. The object of the puzzle is to find a sequence of moves that allow the knight to visit every square on the board exactly once. One such sequence is called a “tour.” The knight’s tour puzzle has fascinated chess players, mathematicians and computer scientists alike for many years. The upper bound on the number of possible legal tours for an eight-by-eight chessboard is known to be $1.305×10^{35}$; however, there are even more possible dead ends. Clearly this is a problem that requires some real brains, some real computing power, or both.

Although researchers have studied many different algorithms to solve the knight’s tour problem, a graph search is one of the easiest to understand and program. Once again we will solve the problem using two main steps:

- Represent the legal moves of a knight on a chessboard as a graph.
- Use a graph algorithm to find a path of length $rows×columns−1$ where every vertex on the graph is visited exactly once.



## Building the Knight’s Tour Graph

To represent the knight’s tour problem as a graph we will use the following two ideas: Each square on the chessboard can be represented as a node in the graph. Each legal move by the knight can be represented as an edge in the graph. The next Figure illustrates the legal moves by a knight and the corresponding edges in a graph.

![](./images/kt1.png)

To build the full graph for an n-by-n board we can use the Python function shown below. The `knightGraph` function makes one pass over the entire board. At each square on the board the `knightGraph` function calls a helper, `genLegalMoves`, to create a list of legal moves for that position on the board. All legal moves are then converted into edges in the graph. Another helper function `posToNodeId` converts a location on the board in terms of a row and a column into a linear vertex number similar to the vertex numbers shown in the previous Figure.

In [1]:
from utils import Graph

def knightGraph(bdSize):
    ktGraph = Graph()
    for row in range(bdSize):
        for col in range(bdSize):
            nodeId = posToNodeId(row,col,bdSize)
            newPositions = genLegalMoves(row,col,bdSize)
            for e in newPositions:
                nid = posToNodeId(e[0],e[1],bdSize)
                ktGraph.addEdge(nodeId,nid)
    return ktGraph

def posToNodeId(row, column, board_size):
    return (row * board_size) + column

The `genLegalMoves` function (Listing below) takes the position of the knight on the board and generates each of the eight possible moves. The `legalCoord` helper function makes sure that a particular move that is generated is still on the board.

In [2]:
def genLegalMoves(x,y,bdSize):
    newMoves = []
    moveOffsets = [(-1,-2),(-1,2),(-2,-1),(-2,1),
                   ( 1,-2),( 1,2),( 2,-1),( 2,1)]
    for i in moveOffsets:
        newX = x + i[0]
        newY = y + i[1]
        if legalCoord(newX,bdSize) and \
                        legalCoord(newY,bdSize):
            newMoves.append((newX,newY))
    return newMoves

def legalCoord(x,bdSize):
    if x >= 0 and x < bdSize:
        return True
    else:
        return False

The next Figure shows the complete graph of possible moves on an eight-by-eight board. There are exactly 336 edges in the graph. Notice that the vertices corresponding to the edges of the board have fewer connections (legal moves) than the vertices in the middle of the board. Once again we can see how sparse the graph is. If the graph was fully connected there would be 4,096 edges. Since there are only 336 edges, the adjacency matrix would be only 8.2 percent full.

<img src="./images/kt2.png" width=640>

## Implementing Knight’s Tour

The search algorithm we will use to solve the knight’s tour problem is called **depth first search (DFS)**. Whereas the breadth first search algorithm discussed in the previous lecture builds a search tree one level at a time, a depth first search creates a search tree by exploring one branch of the tree as deeply as possible. In this section we will look at two algorithms that implement a depth first search. The first algorithm we will look at directly solves the knight’s tour problem by explicitly forbidding a node to be visited more than once. The second implementation is more general, but allows nodes to be visited more than once as the tree is constructed. The second version is used in subsequent sections to develop additional graph algorithms.

The depth first exploration of the graph is exactly what we need in order to find a path that has exactly 63 edges. We will see that when the depth first search algorithm finds a dead end (a place in the graph where there are no more moves possible) it backs up the tree to the next deepest vertex that allows it to make a legal move.

The `knightTour` function takes four parameters: `n`, the current depth in the search tree; `path`, a list of vertices visited up to this point; `u`, the vertex in the graph we wish to explore; and `limit` the number of edges in the path. The `knightTour` function is recursive (i.e. it uses a stack implicitly). When the `knightTour` function is called, it first checks the base case condition. If we have a path that contains 63 vertices for a $8 \times 8$ board, we return from `knightTour` with a status of `True` indicating that we have found a successful tour and the actual knight's tour through the board. If the path is not long enough we continue to explore one level deeper by choosing a new vertex to explore and calling `knightTour` recursively for that vertex.

DFS also uses colors to keep track of which vertices in the graph have been visited. Unvisited vertices are colored white, and visited vertices are colored gray. If all neighbors of a particular vertex have been explored and we have not yet reached our goal length of 64 vertices, we have reached a dead end. When we reach a dead end we must backtrack. Backtracking happens when we return from `knightTour` with a status of `False`. In the breadth first search we used a queue to keep track of which vertex to visit next. Since depth first search is recursive, we are implicitly using a stack to help us with our backtracking. When we return from a call to `knightTour` with a status of `False`, we remain inside the `while` loop and look at the next vertex in `nbrList`.

In [12]:
from utils import Graph, Vertex

def knightTour(n,path,u,limit):
    u.setColor('gray')
    path.append(u)
    if n < limit:
        nbrList = list(u.getConnections())
        i = 0
        done = False
        while i < len(nbrList) and not done:
            if nbrList[i].getColor() == 'white':
                done = knightTour(n+1, path, nbrList[i], limit)
            i = i + 1
        if not done:  # prepare to backtrack
            path.pop()
            u.setColor('white')
    else:
        done = True
        return path
    return done

Let's generate a graph for a board of size only $4 \times 4$ in order to keep the computational cost low:

In [13]:
g = knightGraph(4)
v0 = g.getVertex(0) #Retrieve Vertex with ID 0

Now let's search for a knight's tour:

In [16]:
path=knightTour(0,[],v0,15)
path

False

A tour was not found. The board is too small and the largest path is of lenght 14. Let's try a larger board: $5 \times 5$

In [18]:
g = knightGraph(5)
v0 = g.getVertex(0)
path=knightTour(0,[],v0,24)

In [19]:
for vertex in path:
    print(vertex.id)

0
7
4
13
6
3
14
23
16
5
2
9
12
15
22
19
8
1
10
21
18
11
20
17
24


Let's look at the $5 \times 5$ board with vertices IDs in black and the knight's tour order found by **dfs** in red:

<img src="./images/ktt.png" width=30%>

Let’s look at a simple example of `knightTour` in action. You can refer to the figures below to follow the steps of the search. For this example we will assume that the call to the `getConnections` method on line 6 orders the nodes in alphabetical order. We begin by calling `knightTour(0,path,A,5)`

`knightTour` starts with node A as seen in next Figure a). The nodes adjacent to A are B and D. Since B is before D alphabetically, DFS selects B to expand next as shown in Figure b). Exploring B happens when `knightTour` is called recursively. B is adjacent to C and D, so `knightTour` elects to explore C next. However, as you can see in Figure c) node C is a dead end with no adjacent white nodes. At this point we change the color of node C back to white. The call to `knightTour` returns a value of `False`. The return from the recursive call effectively backtracks the search to vertex B, see Figure d). The next vertex on the list to explore is vertex D, so `knightTour` makes a recursive call moving to node D, see Figure e). From vertex D on, `knightTour` can continue to make recursive calls until we get to node C again, see Figure f), Figure g), and Figure h). However, this time when we get to node C the test `n < limit` fails so we know that we have exhausted all the nodes in the graph. At this point we can return `True` to indicate that we have made a successful tour of the graph. When we return the list, `path` has the values `[A,B,D,E,F,C]`, which is the the order we need to traverse the graph to visit each node exactly once.

![](./images/kt3.png)

The next Figure shows you what a complete tour around an eight-by-eight board looks like. There are many possible tours; some are symmetric. With some modification you can make circular tours that start and end at the same square.

![](./images/kt4.png)

## Knight’s Tour Analysis

There is one last interesting topic regarding the knight’s tour problem, then we will move on to the general version of the depth first search. The topic is performance. In particular, `knightTour` is very sensitive to the method you use to select the next vertex to visit. For example, on a five-by-five board you can produce a path in about 1.5 seconds on a reasonably fast computer. But what happens if you try an $8 \times 8$ board? In this case, depending on the speed of your computer, you may have to wait up to a half hour to get the results! Even a $6 \times 6$ is quite slow:

In [20]:
g = knightGraph(6)
v0 = g.getVertex(0)
path=knightTour(0,[],v0,35)

The reason for this is that the knight’s tour problem as we have implemented it so far is an exponential algorithm of size $O(k^N)$, where $N$ is the number of squares on the chess board, and $k$ is a small constant. The next Figure can help us visualize why this is so. The root of the tree represents the starting point of the search. From there the algorithm generates and checks each of the possible moves the knight can make. As we have noted before the number of moves possible depends on the position of the knight on the board. In the corners there are only two legal moves, on the squares adjacent to the corners there are three and in the middle of the board there are eight. The Figure on the bottom right shows the number of moves possible for each position on a board. At the next level of the tree there are once again between 2 and 8 possible next moves from the position we are currently exploring. The number of possible positions to examine corresponds to the number of nodes in the search tree.


<img src="./images/performance.png" width=70%>

We have already seen that the number of nodes in a binary tree of height $N$ is $2^{N+1}−1$. For a tree with nodes that may have up to eight children instead of two the number of nodes is much larger. Because the branching factor of each node is variable, we could estimate the number of nodes using an average branching factor. The important thing to note is that this algorithm is exponential: $k^{N+1}−1$, where $k$ is the average branching factor for the board. Let’s look at how rapidly this grows! For a board that is 5x5 the tree will be 25 levels deep, or $N = 24$ counting the first level as level 0. The average branching factor is $k=3.8$ So the number of nodes in the search tree is $3.8^{25}−1$ or $3.12×10^{14}$. For a 6x6 board, $k=4.4$, there are $1.5×10^{23}$ nodes, and for a regular $8x8$ chess board, $k=5.25$, there are $1.3×10^{46}$. Of course, since there are multiple solutions to the problem we won’t have to explore every single node, but the fractional part of the nodes we do have to explore is just a constant multiplier which does not change the exponential nature of the problem.

Luckily there is a way to speed up the eight-by-eight case so that it runs in under one second. The listing below shows the code that speeds up the `knightTour`. This function (see Listing below), called `orderbyAvail` will be used in place of the call to `u.getConnections` in the code previously shown above. The critical line in the `orderByAvail` function is line 12. This line ensures that we select the vertex to go next that has the fewest available moves. You might think this is really counter productive; why not select the node that has the most available moves? You can try that approach easily by running the program yourself and inserting the line `resList.reverse()` right after the sort.

The problem with using the vertex with the most available moves as your next vertex on the path is that it tends to have the knight visit the middle squares early on in the tour. When this happens it is easy for the knight to get stranded on one side of the board where it cannot reach unvisited squares on the other side of the board. On the other hand, visiting the squares with the fewest available moves first pushes the knight to visit the squares around the edges of the board first. This ensures that the knight will visit the hard-to-reach corners early and can use the middle squares to hop across the board only when necessary. Utilizing this kind of empirical knowledge to speed up an algorithm is called a **heuristic**. Humans use heuristics every day to help make decisions, heuristic searches are often used in the field of artificial intelligence. This particular heuristic is called Warnsdorff’s algorithm, named after H. C. Warnsdorff who published his idea in 1823.

In [41]:
from utils import Graph, Vertex

def orderByAvail(n):
    resList = []
    for v in n.getConnections():
        if v.getColor() == 'white':
            c = 0
            for w in v.getConnections():
                if w.getColor() == 'white':
                    c = c + 1
            resList.append((c,v))
    resList.sort(key=lambda x: x[0])
    return [y[1] for y in resList]

def knightTourOptimized(n,path,u,limit):
    u.setColor('gray')
    path.append(u)
    if n < limit:
        #nbrList = list(u.getConnections()) # Previous implementation
        nbrList = orderByAvail(u)
        i = 0
        done = False
        while i < len(nbrList) and not done:
            if nbrList[i].getColor() == 'white':
                done = knightTour(n+1, path, nbrList[i], limit)
            i = i + 1
        if not done:  # prepare to backtrack
            path.pop()
            u.setColor('white')
    else:
        done = True
        return path
    return done

Notice the huge improvement in the speed. Now you can compute knight's tours for an $8 \times 8$ board very fast:

In [42]:
g = knightGraph(8)
v0 = g.getVertex(0)
path=knightTourOptimized(0,[],v0,63)

In [43]:
for v in path:
    print(v.id,end=",")

0,10,4,14,31,46,63,53,47,62,52,58,48,33,16,1,11,5,15,21,6,23,38,55,61,51,57,40,25,8,2,17,32,42,59,49,43,60,50,56,41,24,9,26,36,30,20,3,13,7,22,37,27,12,18,28,34,19,29,35,45,39,54,44,

## General Depth First Search

The knight’s tour is a special case of a depth first search where the goal is to create the deepest depth first tree, without any branches. The more general depth first search is actually easier. Its goal is to search as deeply as possible, connecting as many nodes in the graph as possible and branching where necessary.

It is even possible that a depth first search will create more than one tree. When the depth first search algorithm creates a group of trees we call this a **depth first forest**. As with the breadth first search our depth first search makes use of predecessor links to construct the tree. In addition, the depth first search will make use of two additional instance variables in the `Vertex` class. The new instance variables are the discovery and finish times. The discovery time tracks the number of steps in the algorithm before a vertex is first encountered. The finish time is the number of steps in the algorithm before a vertex is colored black. As we will see after looking at the algorithm, the discovery and finish times of the nodes provide some interesting properties we can use in later algorithms.

The code for our depth first search is shown in the next Listing. Since the two functions `dfs` and its helper `dfsvisit` use a variable to keep track of the time across calls to `dfsvisit`, the code is implemented as methods of a class that inherits from the `Graph` class. This implementation extends the graph class by adding a `time` instance variable and the two methods `dfs` and `dfsvisit`. Looking at line 12 you will notice that the `dfs` method iterates over all of the vertices in the graph calling `dfsvisit` on the nodes that are white. The reason we iterate over all the nodes, rather than simply searching from a chosen starting node, is to make sure that all nodes in the graph are considered and that no vertices are left out of the depth first forest. It may look unusual to see the statement `for aVertex in self`, but remember that in this case self is an instance of the `DFSGraph` class, and iterating over all the vertices in an instance of a graph is a natural thing to do.

In [120]:
from utils import Graph, Vertex

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)

Although our implementation of `bfs` was only interested in considering nodes for which there was a path leading back to the start, it is possible to create a breadth first forest that represents the shortest path between all pairs of nodes in the graph. 

The `dfsvisit` method starts with a single vertex called `startVertex` and explores all of the neighboring white vertices as deeply as possible. If you look carefully at the code for `dfsvisit` and compare it to breadth first search, what you should notice is that the `dfsvisit` algorithm is almost identical to `bfs` except that on the last line of the inner `for` loop, `dfsvisit` calls itself recursively to continue the search at a deeper level, whereas `bfs` adds the node to a queue for later exploration. It is interesting to note that where `bfs` uses a queue, `dfsvisit` uses a stack. You don’t see a stack in the code, but it is implicit in the recursive call to `dfsvisit`.

The following sequence of figures illustrates the depth first search algorithm in action for a small graph. In these figures, the dotted lines indicate edges that are checked, but the node at the other end of the edge has already been added to the depth first tree. In the code this test is done by checking that the color of the other node is non-white.

The search begins at vertex A of the graph. Since all of the vertices are white at the beginning of the search the algorithm visits vertex A. The first step in visiting a vertex is to set the color to gray, which indicates that the vertex is being explored and the discovery time is set to 1. Since vertex A has two adjacent vertices (B, D) each of those need to be visited as well. We’ll make the arbitrary decision that we will visit the adjacent vertices in alphabetical order.

Vertex B is visited next, so its color is set to gray and its discovery time is set to 2. Vertex B is also adjacent to two other nodes (C, D) so we will follow the alphabetical order and visit node C next.

Visiting vertex C brings us to the end of one branch of the tree. After coloring the node gray and setting its discovery time to 3, the algorithm also determines that there are no adjacent vertices to C. This means that we are done exploring node C and so we can color the vertex black, and set the finish time to 4. 

Since vertex C was the end of one branch we now return to vertex B and continue exploring the nodes adjacent to B. The only additional vertex to explore from B is D, so we can now visit D and continue our search from vertex D. Vertex D quickly leads us to vertex E. Vertex E has two adjacent vertices, B and F. Normally we would explore these adjacent vertices alphabetically, but since B is already colored gray the algorithm recognizes that it should not visit B since doing so would put the algorithm in a loop! So exploration continues with the next vertex in the list, namely F.

Vertex F has only one adjacent vertex, C, but since C is colored black there is nothing else to explore, and the algorithm has reached the end of another branch. From here on, you will see that the algorithm works its way back to the first node, setting finish times and coloring vertices black.

![](./images/dfs0.png)

Let's verify these results with actual code:

In [138]:
g = DFSGraph()

g.addEdge('A','B')
g.addEdge('A','D')
g.addEdge('B','C')
g.addEdge('B','D')
g.addEdge('D','E')
g.addEdge('E','B')
g.addEdge('E','F')
g.addEdge('F','C')

g.dfs()

for v in g:
    print("Node: " + v.id + ". Color: " + v.color +  ". Discovery time: " + str(v.disc) + ". Finished time: " + str(v.fin))

Node: A. Color: black. Discovery time: 1. Finished time: 12
Node: B. Color: black. Discovery time: 2. Finished time: 11
Node: D. Color: black. Discovery time: 5. Finished time: 10
Node: C. Color: black. Discovery time: 3. Finished time: 4
Node: E. Color: black. Discovery time: 6. Finished time: 9
Node: F. Color: black. Discovery time: 7. Finished time: 8


The starting and finishing times for each node display a property called the **parenthesis property**. This property means that all the children of a particular node in the depth first tree have a later discovery time and an earlier finish time than their parent. The next Figure shows the tree constructed by the depth first search algorithm.

![](./images/dfs.png)

### Depth First Search Analysis

The general running time for depth first search is as follows. The loops in `dfs` both run in $O(V)$, not counting what happens in `dfsvisit`, since they are executed once for each vertex in the graph. In `dfsvisit` the loop is executed once for each edge in the adjacency list of the current vertex. Since `dfsvisit` is only called recursively if the vertex is white, the loop will execute a maximum of once for every edge in the graph or $O(E)$. So, the total time for depth first search is $O(V+E)$.

#### References

- [Problem Solving with Algorithms and Data Structures using Python by Bradley N. Miller, David L. Ranum is licensed under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.](https://runestone.academy/runestone/books/published/pythonds/Graphs/toctree.html)